# Analisador de Fake News


///////////////// Introdução
//////////////// Sumário
# OLHAR ESSES LINKS****
- seguir esse https://towardsdatascience.com/a-complete-exploratory-data-analysis-and-visualization-for-text-data-29fb1b96fb6a - ESSE DAQUI TEM O PRECISION/RECALL/ TRATAMENTO DOS DADOS DO INICIO AO FIM OLHAR ELE: https://stackabuse.com/text-classification-with-python-and-scikit-learn/



1. Importações
2. Leitura dos Dados
3. Limpeza e Transformações dos Dados
4. Redução de Dimensionalidade dos Dados
5. Visualizações dos dados
6. Avaliação dos Modelos
7. Conclusão

# 1. Importações

In [13]:
import numpy as np
import pandas as pd
import warnings
from statistics import mean
from math import log, pi, sqrt
import matplotlib.pyplot as plt
# Tratamento do Texto
from textblob import TextBlob, Word
import nltk
from nltk.corpus import stopwords

from scipy.linalg import svd 
from sklearn import linear_model, datasets, svm, metrics
from scipy.stats import multivariate_normal
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import DistanceMetric, KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop = stopwords.words('english')
warnings.filterwarnings("ignore")
np.random.seed(12345)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 2. Leitura dos Dados

In [14]:
dataset = pd.read_csv("data/data.csv") # será utilizado para computar coisas nas visualizações
dataset['Body'] = dataset['Body'].astype(str) 
data = dataset # será limpado e utilizado nos modelos
data.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


# 3. Limpeza dos Dados

In [15]:
# Deixar tudo em lower case
data['Headline'] = data['Headline'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['Body'] = data['Body'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
# Removendo pontuação
data['Headline'] = data['Headline'].str.replace('[^\w\s]','')
data['Body'] = data['Body'].str.replace('[^\w\s]','')
# Removendo stopwords
data['Headline'] = data['Headline'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data['Body'] = data['Body'].apply(lambda x: " ".join(x for x in str(x).split() if str(x) not in stop))
# 10 Palavras mais frequentes e menos frequentes
freqHeadline = pd.Series(' '.join(data['Headline']).split()).value_counts()[:10]
freqBody = pd.Series(' '.join(data['Body']).split()).value_counts()[:10]
rareHead = pd.Series(' '.join(data['Headline']).split()).value_counts()[-10:]
rareBody = pd.Series(' '.join(data['Body']).split()).value_counts()[-10:]
# Remoção das palavras raras e frequentes
freqHeadline = list(freqHeadline.index)
freqBody = list(freqBody.index)
data['Headline'] = data['Headline'].apply(lambda x: " ".join(x for x in x.split() if x not in freqHeadline))
data['Body'] = data['Body'].apply(lambda x: " ".join(x for x in x.split() if x not in freqBody))
data['Headline'] = data['Headline'].apply(lambda x: " ".join(x for x in x.split() if x not in rareHead))
data['Body'] = data['Body'].apply(lambda x: " ".join(x for x in x.split() if x not in rareBody))
# Lemmatization. Extrai o núcleo/root da palavra
data['Headline'] = data['Headline'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['Body'] = data['Body'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,four way bob corker skewered donald,image copyright getty image sunday morning don...,1
1,https://www.reuters.com/article/us-filmfestiva...,linklaters war veteran comedy speaks modern am...,london reuters last flag flying comedydrama vi...,1
2,https://www.nytimes.com/2017/10/09/us/politics...,trump fight corker jeopardizes legislative agenda,feud broke public view last week mr corker mr ...,1
3,https://www.reuters.com/article/us-mexico-oil-...,egypt cheiron win tieup pemex mexican onshore ...,mexico city reuters egypt cheiron holding limi...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,jason aldean open snl tribute,country singer jason aldean performing la vega...,1


In [16]:
vectorizer = CountVectorizer()
vectorizer.fit(data['Headline']) # tokenize and build vocabulary
vector = vectorizer.transform(data['Headline']) # encode document
print("Tamanho: ", vector.shape)
head = [key for key in  vectorizer.vocabulary_]
df = pd.DataFrame(vector.toarray(), columns = head)
df['Label'] = data['Label']
df.head()

Tamanho:  (4009, 6632)


,four,way,bob,corker,skewered,donald,linklaters,war,veteran,comedy,...,tragic,hervé,leroux,léger,bandage,dress,30minute,100000,antiamerican,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# 4. Redução da Dimensionalidade

In [17]:
from scipy.linalg import svd 

def fisher_score(x, y):
    mean = np.mean(x, axis=0)
    classes = np.unique(y)
    sS = 0
    sD = 0
    Nk = []
    
    meanD = []
    varD = []
    
    for k in classes:
        elements = []
        s = 0
        for i in range(0, len(y)):
            if(y[i] == k):
                s += 1
                elements.append(x[i])
        meanD.append(np.mean(elements, axis=0))
        varD.append(np.var(elements, axis=0))
        Nk.append(s)
    
    for k in range(0, len(classes)):
        sS += (Nk[k] * ((meanD[k] - mean)**2))
        sD += (Nk[k] * varD[k])
    return sS/sD


def compute(x):
    mean = np.mean(x, axis = 0)
    cov = np.cov(np.transpose(x))

    U, S, V = svd(cov)
    S = np.diag(S)
    M = U @ S @ V
    P = np.transpose(U)
    return {'S': S, 'U': U, 'V': V, 'M': M, 'P': P}

def transform(x, rateVariance):
    pca_result = compute(x)
    
    S = pca_result['S']
    P = pca_result['P']
    
 #   autovalores = np.sort([S[i][i] for i in range(len(S))])[::-1]
    #varianciaExplicadaTot = np.sum(np.array(autovalores))
    #varianceExpl = rateVarience*varianciaExplicadaTot

    #matrizTransform = []
    
    matrizTransform = []
    varianciaExplicadaTot = 0
    varianceExpl = 0
    autoValores = []
    for i in range(0, S.shape[0]):
        varianciaExplicadaTot += S[i,i]
        autoValores.append(S[i,i])
    varianceExpl = rateVariance *varianciaExplicadaTot

    autoValOrdenados = np.sort(autoValores)
    autoValOrdenados = autoValOrdenados[::-1]
    
    varianceAtual = 0
    i = 0
    while(varianceAtual<=varianceExpl and i<autoValOrdenados.shape[0]):
        for j in range (0, len(autoValores)):
            if(autoValOrdenados[i]==autoValores[j]):
                matrizTransform.append(P[j])
        varianceAtual += autoValOrdenados[i]
        i += 1
    print(matrizTransform)
    return {'P': matrizTransform, 'Z':np.transpose(np.dot(matrizTransform, np.transpose(x)))}

In [18]:
teste = transform(df, 0.9)

[array([-5.19070031e-04,  4.03976021e-03,  2.05923825e-03, ...,
       -1.87862083e-04,  4.58524291e-04, -9.74390021e-01]), array([ 7.77426522e-06,  4.91272237e-02,  1.00644620e-02, ...,
       -4.97181894e-04, -3.75489104e-04,  6.91696627e-02]), array([-5.73718728e-05, -1.11356675e-02, -2.21988705e-03, ...,
        5.13448452e-04,  6.92393822e-04, -8.08652756e-02]), array([ 2.55097351e-05, -1.66775157e-02,  1.65404412e-03, ...,
        9.78239180e-04, -2.53897859e-03, -5.10269146e-02]), array([-9.37716175e-06,  4.96203031e-02, -1.44920334e-02, ...,
        1.23699834e-04, -2.71625011e-04, -1.83553947e-02]), array([ 5.19699172e-05, -9.93299458e-03, -1.55012149e-02, ...,
       -2.84399667e-04,  1.19527428e-05,  3.45618066e-02]), array([-8.79898468e-05,  1.29833160e-02, -4.52792980e-04, ...,
        1.71046016e-04,  2.25239637e-04,  2.65814265e-02]), array([ 5.44497307e-05, -5.12384652e-03,  4.33649340e-03, ...,
        2.22518843e-04,  1.63709114e-04, -2.39660918e-02]), array([-8.70801

In [19]:
data_pca = pd.DataFrame(teste['Z'])
data_pca.head()

,0,1,2,3,4,5,6,7,8,9,...,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420
0,-0.984137,0.099148,-0.061146,-0.011572,-0.052454,0.014199,0.007428,0.038479,-0.000603,-0.030661,...,0.004141,0.013817,0.004323,-0.016635,0.015530,0.016449,-0.033599,0.044122,0.003906,0.005178
1,-0.997900,0.049870,-0.078135,-0.042030,-0.021946,0.017339,0.017266,-0.015887,0.026235,-0.003166,...,-0.007505,-0.005000,-0.000359,0.016936,-0.005377,0.027472,-0.003147,-0.031680,0.032852,-0.040711
2,-0.980638,0.041407,-0.039074,0.015742,0.003620,-0.018254,0.056485,0.000034,-0.014970,-0.105377,...,0.027347,-0.087965,0.065068,0.052900,-0.022578,0.014695,0.039341,-0.073220,0.031420,-0.030568
3,-0.946827,0.820681,-0.232655,-0.163426,0.050937,-0.533305,0.017323,-0.147484,0.209413,-0.264276,...,-0.045189,0.039284,0.039805,0.032814,0.046246,0.074917,-0.010465,-0.032566,0.002511,0.012827
4,-1.008233,0.063518,-0.075735,-0.032143,-0.019035,0.027909,0.067612,-0.072380,-0.040555,0.013342,...,0.006500,-0.000961,-0.030848,-0.004062,0.016977,0.014336,0.010875,0.007983,0.001482,-0.008184


In [20]:
fisher_scores = fisher_score(df.iloc[:,:-1].values, df['Label'].values)
qnt = 1000 # quantas colunas quer dropar pelo fisher score
indices = [] # armazena os indices dos menores valores do fisher_score -> que é o que vamos dropar
for i in range(qnt): # faz o processamento de armazenar os indices dos menores valores
    indice = np.argmin(fisher_scores)
    fisher_scores = np.delete(fisher_scores, indice)
    indices.append(indice)
data_fisher = df.drop([df.columns[i] for i in indices] ,  axis='columns')
data_fisher.head()

,four,way,bob,corker,skewered,donald,linklaters,war,veteran,comedy,...,tragic,hervé,leroux,léger,bandage,dress,30minute,100000,antiamerican,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# 5. Visualização e Análise dos dados

- https://www.mien.in/2017/10/02/visual-text-analytics-with-python/
- visualizações ta tudo aqui: https://towardsdatascience.com/a-complete-exploratory-data-analysis-and-visualization-for-text-data-29fb1b96fb6a
- o coisa la do kaggle https://www.kaggle.com/kumudchauhan/fake-news-analysis-and-classification#506957

0. dataset.describe()
0. dataset.corr()

0. WordCloud

1. Palavras mais frequentes por classe - sem stopwords - (grafico de barras)
2. Palavras menos frequentes por classe - sem stopwords - (grafico de barras)

3. Relação de stopwords em noticias reais/Fakes
4. Relação da Escrita errada/correta em noticias reais/Fakes

5. The distribution of top bigrams before removing stop words
6. Analise de Sentimentos

Which sources publish real news
Which sources publish maximum fake news?
3 Is there any common source which reports both real and fake news?
 Do sources include movies in the news to get more attention?
 Do sources include images in all the news as images provides visual cues?
 Most discriminatory words in the title of news
 Density distribuiton of title length for real and fake news


Lexical dispersion plot
Frequency distribution plot:
Lexical diversity dispersion plot
N-gram frequency distribution plot:
Top 20 bigrams in review after removing stop words
The distribution of Top trigrams before removing stop words
The distribution of Top trigrams after removing stop words

In [21]:
# https://pythondata.com/text-analytics-visualization/
# https://doc.lagout.org/Others/Data%20Mining/Text%20Mining%20and%20Visualization_%20Case%20Studies%20using%20Open-Source%20Tools%20%5BHofmann%20%26%20Chisholm%202015-12-18%5D.pdf
# https://itnext.io/basics-of-text-analysis-visualization-1978de48af47
# https://kanoki.org/2019/03/17/text-data-visualization-in-python/
# https://realpython.com/python-keras-text-classification/ (LER ISSO AQUI SEM DELONGAS)
# https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/ (LER ISSO AQUI)
# https://towardsdatascience.com/text-classification-in-python-dd95d264c802 (LER ISSO AQUI)
#
#

In [22]:
data.raw

AttributeError: 'DataFrame' object has no attribute 'raw'

In [ ]:
wordcloud = WordCloud(max_font_size=60).generate(data)
plt.figure(figsize=(16,12))
# plot wordcloud in matplotlib
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# 6. Avaliação de Métodos

In [23]:
# Split dos Dados
# atribuindo X com as features e y com os valores de classe
X = df.iloc[:,:-1].values
Y = df['Label'].values
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25)

## 6.1. Regressão Logística

In [24]:
logreg = linear_model.LogisticRegression(C = 5)
logreg.fit(x_train, y_train)
predictedRL = logreg.predict(x_test)
score = accuracy_score(y_pred = predictedRL, y_true = y_test)
score

0.8424725822532403

## 6.2. KNN

In [31]:
dist = DistanceMetric.get_metric('euclidean')
knn = KNeighborsClassifier(n_neighbors = 3, metric='euclidean')

# treinando o modelo com os conjuntos de treino
knn.fit(x_train, y_train)

# realizando a predicao
predictedKNN = knn.predict(x_test)
score = accuracy_score(y_pred = predictedKNN, y_true = y_test)
score

0.6251246261216351

## 6.3. Análise Discriminante Gaussiano

In [25]:
clf = GaussianNB()
clf.fit(x_train, y_train)
predictedNB = clf.predict(x_test)
score = accuracy_score(y_pred = predictedNB, y_true = y_test)
score

0.7996011964107677

## 6.4. Árvore de Decisão

In [26]:
tree_classifier = DecisionTreeClassifier()
tree_classifier.fit(x_train, y_train)
predictedAD = tree_classifier.predict(x_test)
score = accuracy_score(y_pred = predictedAD, y_true = y_test)
score

0.8354935194416749

## 6.5. MLP

In [27]:
# params: 
    # hidden_layer_sizes 
    # activation : {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
    # alpha
    # batch-size
    # learning_rate
    # momentum

mlp = MLPClassifier(hidden_layer_sizes = (300, 20))
mlp.fit(x_train, y_train)

# realizando a predicao
predictedMLP = mlp.predict(x_test)
score = accuracy_score(y_pred = predictedMLP, y_true = y_test)
score

0.8414755732801595

## 6.1. SVM

In [28]:
#model = svm.SVC(gamma='auto')
#param_grid = [{'kernel': ['rbf'], 'C': 2 ** np.arange(-5.0, 16.0, 2),
#                                  'gamma': 2 ** np.arange(-15.0, 4.0, 2)},
#              {'kernel': ['poly'], 'C': 2 ** np.arange(-5.0, 16.0, 2),
#                                   'degree': np.arange(2, 6)},
#              {'kernel': ['linear'], 'C': 2 ** np.arange(-5.0, 16.0, 2)}]

#model = GridSearchCV(model, param_grid, cv=3, n_jobs=-1)
#model.fit(x_train, y_train) 

#support_vectors = model.best_estimator_.support_vectors_
#support_vectors

## 6.2. Random Forest

In [29]:
num_arvores = 10
max_columns = 3
max_profundidade = 10
clf = RandomForestClassifier(num_arvores,"gini",max_columns,max_profundidade)
clf.fit(x_train, y_train)
predictedRF = clf.predict(x_test)
score = accuracy_score(y_pred = predictedRF, y_true = y_test)
score

0.5702891326021934